# Data

In [3]:
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
DATADIR = 'C:/Users/sai krishna/Desktop/human/Human Activity Recognition from Sensor Data/UCI_HAR_Dataset/'

# Functions

In [5]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [6]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

In [7]:
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'C://UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 
    return np.transpose(signals_data, (1, 2, 0))

In [8]:
def load_y(subset):
    filename = f'C://UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [9]:
def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))


# Defining the neccessary labels

In [11]:
ACTIVITIES = { 0: 'WALKING', 1: 'WALKING_UPSTAIRS', 2: 'WALKING_DOWNSTAIRS', 3: 'SITTING', 4: 'STANDING', 5: 'LAYING'}
SIGNALS = ["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", 
           "total_acc_x", "total_acc_y", "total_acc_z"]

# Loading Data and preparing for Deep Learning input

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

ModuleNotFoundError: No module named 'tensorflow'

In [13]:
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

NameError: name 'tf' is not defined

In [91]:
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [92]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [93]:
epochs = 7
batch_size = 16
n_hidden = 32

In [4]:
import sys
X_train, X_test, Y_train, Y_test = load_data()

NameError: name 'load_data' is not defined

In [96]:
print(X_test[0].shape)

(128, 9)


In [97]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


# An attemt to visualize the data alignment

#### t[9] = for a particular instant of time, we have 9 readings of sensors (3 acc, 3 gyro, 3 total). So that's a 9 dim col matrix. 
#### we have 128 such values per window = w[[t1], [t2], ... , [t128]] 
#### we have 7352 suh windows = timeseries[w1, w2, ... , w7352]

# Defining and Applying LSTM

In [35]:
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [36]:

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Fitting and Testing model

In [37]:
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7352 samples, validate on 2947 samples
Epoch 1/7
7352/7352 [==============================] - 73s 10ms/step - loss: 1.2944 - accuracy: 0.4597 - val_loss: 1.0708 - val_accuracy: 0.5412
Epoch 2/7
7352/7352 [==============================] - 77s 10ms/step - loss: 0.9070 - accuracy: 0.6230 - val_loss: 0.8601 - val_accuracy: 0.5792
Epoch 3/7
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7480 - accuracy: 0.6518 - val_loss: 0.7388 - val_accuracy: 0.6152
Epoch 4/7
7352/7352 [==============================] - 71s 10ms/step - loss: 0.6728 - accuracy: 0.6768 - val_loss: 0.8630 - val_accuracy: 0.6105
Epoch 5/7
7352/7352 [==============================] - 79s 11ms/step - loss: 0.6304 - accuracy: 0.6921 - val_loss: 0.7357 - val_accuracy: 0.6525
Epoch 6/7
7352/7352 [==============================] - 92s 12ms/step - loss: 0.5828 - accuracy: 0.7361 - val_loss: 0.6825 - val_accur

In [14]:
model.save("abc.h5")

NameError: name 'model' is not defined

In [80]:
model1 = tf.keras.models.load_model("abc.h5", compile = False)

In [46]:
Y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [48]:
xtest_out = model1.predict(X_test)

In [49]:
xtest_out

array([[5.3231716e-03, 4.1961670e-04, 3.3605099e-04, 6.3299030e-02,
        6.5700221e-01, 5.1203370e-04],
       [4.5948029e-03, 3.3581257e-04, 2.6914477e-04, 5.9640020e-02,
        7.0317918e-01, 5.3024292e-04],
       [4.6633184e-03, 3.1635165e-04, 2.5251508e-04, 5.8462441e-02,
        7.0664942e-01, 4.9614906e-04],
       ...,
       [4.7271869e-01, 9.6690792e-01, 9.3372440e-01, 7.2181821e-03,
        7.1561337e-03, 2.4232268e-04],
       [2.5994664e-01, 2.1989003e-01, 1.0835287e-01, 1.5335977e-03,
        1.5983582e-03, 4.4286251e-05],
       [2.3485699e-01, 7.5644064e-01, 6.2824970e-01, 1.7850101e-03,
        1.9616252e-03, 1.2474088e-04]], dtype=float32)

In [60]:
Y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [63]:
labels_xtest = np.argmax(xtest_out, axis=-1)    
print(labels_xtest)

[4 4 4 ... 1 0 1]


In [65]:
labels_ytest = np.argmax(Y_test, axis=-1)    
print(labels_ytest)

[4 4 4 ... 1 1 1]


In [76]:
predicted_data = pd.DataFrame(labels_xtest,labels_ytest ).reset_index()

In [78]:
predicted_data

,index,0
0,4,4
1,4,4
2,4,4
3,4,4
4,4,4
5,4,4
6,4,4
7,4,4
8,4,4
9,4,4


In [50]:
print(confusion_matrix(Y_test, model1.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        0        28        0                   0   
SITTING                  0      428        58        3                   0   
STANDING                 0      131       398        1                   1   
WALKING                  0        0         7      439                   0   
WALKING_DOWNSTAIRS       0        0         0        8                   0   
WALKING_UPSTAIRS         0        0         2       49                   9   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           1  
WALKING                           50  
WALKING_DOWNSTAIRS               412  
WALKING_UPSTAIRS                 411  


In [57]:
from keras import optimizers
model1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [58]:
score = model1.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/sample - loss: 0.5826 - acc: 0.7414


In [59]:
score

[0.5825504896132713, 0.74143195]